In [ ]:
from backtester import Backtester

import pymc as pm
import numpy as np 
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import itertools
from math import sqrt
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

stock = 'UAL'


backtester = Backtester(stock, 'SMART', 'USD')

df = backtester.one_yr_1d_data
close = df['close']
data = np.log(df['close']).diff().dropna()

# 70% train, 15% validate, 15% test
# data is time series data, so we should split it by time

train_size = int(len(data) * 0.7)
validate_size = int(len(data) * 0.15)

train_data = data[:train_size]
validate_data = data[train_size:train_size + validate_size]
test_data = data[train_size + validate_size:]

# print size
print('train size:', len(train_data))
print('validate size:', len(validate_data))
print('test size:', len(test_data))


hyperparameters = {'dt': 1/252}


# fit the model with train data
r_hat = np.mean(data)
sigma_hat = np.std(data)
# annuallized
dt = hyperparameters['dt']
mu = r_hat/dt + 0.5 * sigma_hat**2
sigma = sqrt(sigma_hat**2/dt)
# print all these numbers
print(f"r-hat: {r_hat}, sigma-hat: {sigma_hat}")
print(f"mu: {mu}, sigma: {sigma}")


# annualized returns and volatility
annualized_return = r_hat * 252
annualized_volatility = sigma_hat * sqrt(252)

print(f"Annualized Return: {annualized_return}")
print(f"Annualized Volatility: {annualized_volatility}")
print(f"First close: {close.iloc[0]}")
print(f"Last close: {close.iloc[-1]}")

mu_prior = lambda: pm.Normal('mu', mu=mu, sigma=sigma * 0.1)
sigma_scale = np.sqrt(np.pi / 2) * sigma
sigma_prior = lambda: pm.HalfNormal('sigma', sigma=sigma_scale)
likelihood = lambda mu, sigma: pm.Normal('returns', mu=(mu - 0.5 * sigma**2)*dt, sigma=sigma * np.sqrt(dt), observed=train_data)

prior_configs = {
    "model_normal_halfnormal": {
        "mu_prior": mu_prior,
        "sigma_prior": sigma_prior,
        "likelihood": likelihood
    },
    "model_studentt_halfcauchy_5": {
        "mu_prior": lambda: pm.StudentT('mu', nu=5, mu=0, sigma=1),
        "sigma_prior": lambda: pm.HalfCauchy('sigma', beta=0.5),
        "likelihood": lambda mu, sigma: pm.StudentT('returns', nu=5, mu=mu - 0.5 * sigma**2, sigma=sigma, observed=train_data)
    },
    "model_studentt_halfcauchy_10": {
        "mu_prior": lambda: pm.StudentT('mu', nu=10, mu=0, sigma=1),
        "sigma_prior": lambda: pm.HalfCauchy('sigma', beta=0.5),
        "likelihood": lambda mu, sigma: pm.StudentT('returns', nu=10, mu=mu - 0.5 * sigma**2, sigma=sigma, observed=train_data)
    },
    "model_studentt_halfcauchy_30": {
        "mu_prior": lambda: pm.StudentT('mu', nu=30, mu=0, sigma=1),
        "sigma_prior": lambda: pm.HalfCauchy('sigma', beta=0.5),
        "likelihood": lambda mu, sigma: pm.StudentT('returns', nu=30, mu=mu - 0.5 * sigma**2, sigma=sigma, observed=train_data)
    }
}

model_results = {}

for model_name, config in prior_configs.items():
    with pm.Model() as model:
        # Define priors
        mu = config["mu_prior"]()
        sigma = config["sigma_prior"]()

        # Define likelihood
        likelihood = config["likelihood"](mu, sigma)

        # Sample from posterior
        trace = pm.sample(
            draws=1000,
            tune=500,
            target_accept=0.8,
            chains=4,
            cores=4,
            random_seed=42
        )

        # Posterior predictive checks on validation data
        # We'll generate posterior predictive samples conditioned on the fitted parameters.
        # To do this properly, we need to define a predictive model.
        # Since the validation data is out-of-sample, we just simulate what the model would predict.
        # For simplicity, we assume the same model form applies: returns ~ distribution(mu - 0.5*sigma^2, sigma)
        # We'll generate predictions for the validation time indices, even though it doesn't condition on new covariates.
        
        # One approach:
        # We can't directly "observe" validation_data in the same model run. Instead, we can:
        # 1. Extract posterior samples of mu and sigma from 'trace'.
        # 2. For each sample, simulate synthetic validation returns from the same likelihood distribution.
        
        post_mu_samples = trace.posterior['mu'].values.flatten()
        post_sigma_samples = trace.posterior['sigma'].values.flatten()
        
        # Generate posterior predictive samples for validation data:
        # We'll just draw from the predictive distribution:
        # returns ~ LikelihoodDistribution(mu - 0.5 * sigma^2, sigma)
        n_post_samples = len(post_mu_samples)
        n_val = len(validate_data)
        
        # For demonstration, we assume the same process applies to validation:
        # If you're modeling time-series with drift over time, you'd need a time-series forecasting approach.
        predictive_samples = np.empty((n_post_samples, n_val))
        for i in range(n_post_samples):
            mu_pred = post_mu_samples[i]
            sigma_pred = post_sigma_samples[i]
            # Draw from the specified distribution:
            # If Normal: np.random.normal(mu_pred - 0.5 * sigma_pred**2, sigma_pred, size=n_val)
            # If StudentT: add logic for StudentT as well.
            # We'll assume Normal for demonstration:
            # Adjust depending on which model you're checking.
            if "StudentT" in model_name:
                # for StudentT:
                nu = 5  # hard-coded since we used it in priors
                predictive_samples[i, :] = pm.draws_from_random_variable(
                    pm.distributions.continuous.StudentT.dist(nu=nu, mu=mu_pred - 0.5 * sigma_pred**2, sigma=sigma_pred), 
                    draws=n_val,
                    random_seed=(42 + i)
                )
            else:
                # Normal:
                predictive_samples[i, :] = np.random.normal(mu_pred - 0.5 * sigma_pred**2, sigma_pred, size=n_val)

        # Compute an evaluation metric on validation_data
        # For example, Mean Squared Error:
        mean_predictions = predictive_samples.mean(axis=0)
        mse = np.mean((validate_data - mean_predictions)**2)

        model_results[model_name] = {
            "trace": trace,
            "mse": mse
        }
        print(f"{model_name} MSE: {mse}")

# After looping through all models, select the model with the best performance:
best_model = min(model_results, key=lambda k: model_results[k]["mse"])
print(f"Best Model: {best_model} with MSE: {model_results[best_model]['mse']}")




WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Loading historical data from cache...
Loading historical data from cache...
Loading historical data from cache...
train size: 175
validate size: 37
test size: 39
r-hat: 0.0035381385085537344, sigma-hat: 0.027451606159246463
mu: 0.8919876994959023, sigma: 0.43578073791993077
Annualized Return: 0.8916109041555411
Annualized Volatility: 0.4357807379199307
First close: 41.26
Last close: 100.28


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, sigma]


Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 1 seconds.


model_normal_halfnormal MSE: 0.0006262979240132546


...

Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 1 seconds.


model_studentt_halfcauchy_10 MSE: 0.0006898228231714164


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, sigma]


Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 1 seconds.


model_studentt_halfcauchy_30 MSE: 0.000677927546010468
Best Model: model_normal_halfnormal with MSE: 0.0006262979240132546
